# Programa auxiliar para generar transformaciones al azar en las imágenes de Carteles (cada una con su correspondiente XML) aplicando técnicas de "Data Augmentation"
Basado y adaptado del código de https://medium.com/@bhuwanbhattarai/image-data-augmentation-and-parsing-into-an-xml-file-in-pascal-voc-format-for-object-detection-4cca3d24b33b

0) Configurar los parámetros para la ejecución:

In [1]:
# directorios locales donde se guardan las imágenes a procesar y los XMLs (pueden ser el mismo o distinto)
img_path = 'gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/' 
xml_path = img_path

# directorio local donde se guardarán las nuevas imágenes y XMLs generados en forma temporal
out_path = 'gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/DA/'

# directorios locales donde se guardarán las nuevas imágenes generados al final (pueden ser el mismo o distinto)
final_path_img = img_path
final_path_XML = img_path

print ("Parámetros definidos.")

Parámetros definidos.


1) Cargar librerías:

In [2]:
import os
import cv2
import xml.etree.cElementTree as ET
from PIL import Image
import numpy as np
import random
import shutil

print ("Librerías cargadas.")

Librerías cargadas.


2) Montar el Drive:

In [3]:
# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


3) Definir funciones auxiliares para Data Augmentation que se usarán luego:

In [4]:

# función auxiliar para aplicar PCA color augmentation
def da_pca_color(image):

    # aplica la tansformación
    assert image.ndim == 3 and image.shape[2] == 3
    assert image.dtype == np.uint8
    img = image.reshape(-1, 3).astype(np.float32)
    sf = np.sqrt(3.0/np.sum(np.var(img, axis=0)))
    img = (img - np.mean(img, axis=0))*sf 
    cov = np.cov(img, rowvar=False) # calculate the covariance matrix
    value, p = np.linalg.eig(cov) # calculation of eigen vector and eigen value 
    rand = np.random.randn(3)*0.08
    delta = np.dot(p, rand*value)
    delta = (delta*255.0).astype(np.int32)[np.newaxis, np.newaxis, :]
    img_out = np.clip(image+delta, 0, 255).astype(np.uint8)
    
    return img_out


# función auxiliar para aplicar ruido Gaussiano
def da_noiseG(image):

    # aplica la transformación
    row,col,ch= image.shape
    mean = 0
    var = 0.2
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss

    return noisy    


# función auxiliar para aplicar ruido Salt & Pepper
def da_noiseSP(image):

    # aplica la transformación
    prob = 0.05
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]

    return output    


# función auxiliar para voltear la imagen en forma vertical
def da_flip_vertical(image):

    # apica la transformación
    f_img = cv2.flip(image, 0)

    return f_img


# función auxiliar para voltear la imagen en forma horizontal
def da_flip_horizontal(image):

    # apica la transformación
    f_img = cv2.flip(image, 1)

    return f_img


# función auxiliar para rotar la imagen 90 grados
def da_rotate_90(image):

    # apica la transformación
    img_transpose = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose, 1)

    return f_img


# función auxiliar para rotar la imagen 180 grados
def da_rotate_180(image):

    # apica la transformación
    f_img = cv2.flip(image, -1)

    return f_img


# función auxiliar para rotar la imagen 270 grados
def da_rotate_270(image):

    # apica la transformación
    img_transpose_270 = np.transpose(image, (1,0,2))
    f_img = cv2.flip(img_transpose_270, 0)

    return f_img

print ("Funciones de Data Augmentation para imágenes definidas.")

Funciones de Data Augmentation para imágenes definidas.


4) Levantar la información del  XML para  la imágen que se quiere procesar  y generar transformaciones (nuevas imágenes con su XMLs):

In [5]:
# levanta los xml a procesar
all_xml_array = [os.path.join(xml_path, fn) for fn in os.listdir(xml_path) if fn.endswith('.xml') ]

# crea el directorio temporal si no existe
if not os.path.isdir(out_path):
  os.makedirs(out_path)

# procesa los archivos XML
new_xml_info = []
cantXMLProc = 0
cantImgGen = 0
cantXMLGen = 0
print("\n*** Comienza procesamiento de XMLs para generar nuevas imágenes ***\n")
for xml_file in all_xml_array:

    cantXMLProc = cantXMLProc + 1
    print(cantXMLProc, "> ", xml_file)

    # carga la info del XML original
    et = ET.parse(xml_file)
    element = et.getroot()
    element_objs = element.findall('object') 
    element_filename = element.find('filename').text
    img_filename = os.path.join(img_path, element_filename)
    img_split = element_filename.strip().split('.png')

    # carga la imagen
    img = cv2.imread(img_filename)
    alto, ancho, ch = img.shape

    # genera las nuevas imágenes con su correspondiente XML
    # indicar las operaciones disponibles en el vector de abajo
    #        ('c', 'ng', 'nsp', 'fv', 'fh', 'r90', 'r180', 'r270')
    # -- nota: para carteles/letras no tiene sentido voltear ni girar la imagen     
    for i in ('c', 'ng', 'nsp'):

      if i=='c':        
          # aplica la transformación PCA color
          nuevaImg = da_pca_color(img)
          nuevaImgFN = img_split[0] + '-c.png'

      elif i=='ng': 
          # aplica la transformación ruido Gaussiano
          nuevaImg = da_noiseG(img)        
          nuevaImgFN = img_split[0] + '-ng.png'      

      elif i=='nsp':
          # aplica la transformación ruido Salt & Pepper
          nuevaImg = da_noiseSP(img)
          nuevaImgFN = img_split[0] + '-nsp.png'

      elif i=='fv':
          # aplica transformación de volteo vertical
          nuevaImg = da_flip_vertical(img)
          nuevaImgFN = img_split[0] + '-fv.png'

      elif i=='fh':
          # aplica transformación de volteo horizontal 
          nuevaImg = da_flip_horizontal(img)
          nuevaImgFN = img_split[0] + '-fh.png'  

      elif i=='r90':
          # aplica rotación de 90 grados 
          nuevaImg = da_rotate_90(img)
          nuevaImgFN = img_split[0] + '-r90.png'
        
      elif i=='r180':
          # aplica rotación de 180 grados 
          nuevaImg = da_rotate_180(img)
          nuevaImgFN = img_split[0] + '-r180.png'   
      
      else:
          # aplica rotación de 270 grados 
          nuevaImg = da_rotate_270(img)
          nuevaImgFN = img_split[0] + '-r270.png'      

      # si el archivo no existe
      if not os.path.isfile(out_path + nuevaImgFN):

          # graba la nueva imagen       
          cv2.imwrite(out_path + nuevaImgFN, nuevaImg)
          cantImgGen = cantImgGen + 1
      
          # genera el XML de la nueva imagen   
          nuevo_xml_annotation = ET.Element('annotation')
          ET.SubElement(nuevo_xml_annotation, 'folder').text = final_path_img
          ET.SubElement(nuevo_xml_annotation, 'filename').text = nuevaImgFN
          ET.SubElement(nuevo_xml_annotation, 'path').text = final_path_img + nuevaImgFN

          nuevo_xml_source = ET.SubElement(nuevo_xml_annotation, 'source')
          ET.SubElement(nuevo_xml_source, 'database').text = 'Unknown'

          nuevo_xml_size = ET.SubElement(nuevo_xml_annotation, 'size')
          ET.SubElement(nuevo_xml_size, 'width').text = str(nuevaImg.shape[1])
          ET.SubElement(nuevo_xml_size, 'height').text = str(nuevaImg.shape[0])
          ET.SubElement(nuevo_xml_size, 'depth').text = str(nuevaImg.shape[2])

          ET.SubElement(nuevo_xml_annotation, 'segmented').text = '0'

          # procesa los elementos en el archivo XML original para generar el nuevo
          for element_obj in element_objs:

              # obtiene la información actual de la imagen
              class_name = element_obj.find('name').text 

              # obtiene info del box actual
              obj_bbox = element_obj.find('bndbox')
              x1 = int(round(float(obj_bbox.find('xmin').text)))
              y1 = int(round(float(obj_bbox.find('ymin').text)))
              x2 = int(round(float(obj_bbox.find('xmax').text)))
              y2 = int(round(float(obj_bbox.find('ymax').text)))
              
              # cambia las coordenadas del box de acuerdo al tipo de transformación aplicada
              if i=='c' or i=='ng' or i=='nsp':        
                # para la transformación PCA color, ruido Gaussiano y ruido Salt & Pepper  
                # quedan iguales
                f_x1 = x1
                f_x2 = x2
                f_y1 = y1
                f_y2 = y2

              elif i=='fv':
                  # para transformación de volteo vertical, cambia
                  f_x1 = x1
                  f_y1 = alto - y2
                  f_x2 = x2
                  f_y2 = alto - y1

              elif i=='fh':
                  # para transformación de volteo horizontal, cambia
                  f_x1 = ancho - x2
                  f_y1 = y1
                  f_x2 = ancho - x1
                  f_y2 = y2

              elif i=='r90':
                  # para rotación de 90 grados, cambia
                  f_x1 = alto - y2
                  f_y1 = x1
                  f_x2 = alto - y1
                  f_y2 = x2
                
              elif i=='r180':
                  # para rotación de 180 grados, cambia
                  f_x1 = ancho - x2
                  f_y1 = alto - y2
                  f_x2 = ancho - x1
                  f_y2 = alto - y1   
                
              else:
                  # para rotación de 270 grados, cambia
                  f_x1 = y1
                  f_y1 = ancho - x2
                  f_x2 = y2
                  f_y2 = ancho - x1
          
              # define el nuevo box
              nuevo_xml_object = ET.SubElement(nuevo_xml_annotation, 'object')
              ET.SubElement(nuevo_xml_object, 'name').text = class_name
              ET.SubElement(nuevo_xml_object, 'pose').text = 'Unspecified'
              ET.SubElement(nuevo_xml_object, 'truncated').text = '0'
              ET.SubElement(nuevo_xml_object, 'difficult').text = '0'

              nuevo_xml_bndbox = ET.SubElement(nuevo_xml_object, 'bndbox')
              ET.SubElement(nuevo_xml_bndbox, 'xmin').text = str(f_x1)
              ET.SubElement(nuevo_xml_bndbox, 'ymin').text = str(f_y1)
              ET.SubElement(nuevo_xml_bndbox, 'xmax').text = str(f_x2)
              ET.SubElement(nuevo_xml_bndbox, 'ymax').text = str(f_y2)


          # graba el nuevo XML
          nuevoXMLfn = nuevaImgFN.replace(".png", ".xml")
          xml_tree = ET.ElementTree(nuevo_xml_annotation)
          xml_tree.write( out_path + nuevoXMLfn )
          cantXMLGen = cantXMLGen + 1

print("\n*** Fin procesamiento de de XMLs para generar nuevas imágenes ***\n")
print("> Imágenes/XMLs procesados: ", cantXMLProc)
print("> Imágenes nuevas generadas: ", cantImgGen)
print("> XMLs nuevas generados: ", cantXMLGen)



*** Comienza procesamiento de XMLs para generar nuevas imágenes ***

1 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_1_JXFA903.xml
2 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_2_82J.xml
3 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_3_93RUJ8VYIO.xml
4 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_4_HH.xml
5 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_5_F35.xml
6 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_6_77.xml
7 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_7_386Y25UG.xml
8 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_8_NI2M.xml
9 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_9_S2SGH3XF09.xml
10 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_10_0YLFJRSJ3U.xml
11 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_11_K7THI4H.xml
12 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_12_E4CT.xml
13 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carte

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


230 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_230_EW6PJ.xml
231 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_231_W4.xml
232 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_232_AUPI55I.xml
233 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_233_2I845A.xml
234 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_234_ONOW4A7S.xml
235 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_235_A39TOL81R.xml
236 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_236_6RML.xml
237 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_237_DRY.xml
238 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_238_H9HBCM6.xml
239 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_239_JQ.xml
240 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_240_A32.xml
241 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_241_A54H82GXKP.xml
242 >  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/cartel_242_INC.xml
243 >  g

5) Mover las imágenes nuevas generasdas con su XMLs al carpeta final para su procesamiento:

In [6]:
# levanta la lista de imágenes de cada letra
allFileNames = [ fn for fn in os.listdir(out_path) if fn.endswith('.png') ]

cantIm = 0
cantXML = 0

if not os.path.isdir(final_path_img):
  os.makedirs(final_path_img)

if not os.path.isdir(final_path_XML):
  os.makedirs(final_path_XML)

for name in allFileNames:
  # mueve la imagen
  if os.path.isfile(final_path_img + '/' + name):
    print("**", name, " ya existe en ", final_path_img)      
  else:
    shutil.move(out_path + '/' + name, final_path_img)
    cantIm = cantIm + 1
  # mueve el xml
  nameXML = name.replace(".png", ".xml")
  if os.path.isfile(final_path_XML + '/' + nameXML):
    print("**", nameXML, " ya existe en ", final_path_XML)      
  else:
    shutil.move(out_path + '/' + nameXML, final_path_XML)
    cantXML = cantXML + 1

print(cantIm, " imágenes movidas a ", final_path_img)
print(cantXML, " XML movidos a ", final_path_XML)
print("\n")

1500  imágenes movidas a  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/
1500  XML movidos a  gdrive/My Drive/IA/demoObjDet-Carteles/Carteles/


